<a href="https://colab.research.google.com/github/RodrigoEduard/Codigos-TFM-VIU-2023/blob/Analisis_y_Visualizacion_de_Secuencias_Zona_Austral/TFM_VIU_2023_35_zona_austral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ZONA 5 (AUSTRAL): XI y XII: 9
#   DE AISEN DEL GRAL. C. IBAÑEZ DEL CAMPO(5), DE MAGALLANES Y DE LA ANTARTICA CHILENA(4)

library(readxl)
ZAU <- read_excel("05-ZonaSurAustral.xlsx")
ZAU <- as.data.frame(ZAU)
rownames(ZAU) <-  ZAU$id
# DATA >*************************Data***********************************************
library(TraMineR)
seqstatl(ZAU[, 1:97])

ZAU.alphabet <- c("SOBRE","IGUAL","BAJO","SIN_REG")
ZAU.labels <- c("SOBRE","IGUAL","BAJO","SIN_REG")
ZAU.scodes <- c("SO", "IG", "BA", "SR")
ZAU.seq <- seqdef(ZAU, 2:97, alphabet = ZAU.alphabet, states = ZAU.scodes,
                  labels = ZAU.labels, xtstep = 4)
colores <- c( "#D98880","#2980B9","#73C6B6","#EB984E")  #Rojo, Azul, Verde, Naranja   https://htmlcolorcodes.com/

# Visualización de conjuntos de datos **********Visualize********************************
#pdf("graf1.pdf", height = 10, width = 10)
dev.new()
par(mfrow = c(1, 1))
par(mar = c(4.0, 7.0, 4.0, 4.0)) #inferior, izquierdo, superior y derecho

seqIplot(ZAU.seq, with.legend = FALSE, border = "black", ytlab = "id", ylas = 1, cex.axis = 0.5, xtlab = FALSE, cpal = colores)
text(x = seq_along(colnames(ZAU.seq)), y = par("usr")[3] - 0.3, labels = colnames(ZAU.seq), srt = 90, adj = c(1, 0.5), xpd = TRUE, cex = 0.7)

seqIplot(ZAU.seq, sortv = "from.start", with.legend = FALSE, border = "black", ytlab = "id", ylas = 1, cex.axis = 0.6, xtlab = FALSE, cpal = colores)
text(x = seq_along(colnames(ZAU.seq)), y = par("usr")[3] - 0.3, labels = colnames(ZAU.seq), srt = 90, adj = c(1, 0.5), xpd = TRUE, cex = 0.7)

seqfplot(ZAU.seq, with.legend = FALSE, border = "black", cpal = colores)
text(x = seq_along(colnames(ZAU.seq)), y = par("usr")[3] - 0.3, labels = colnames(ZAU.seq), srt = 90, adj = c(1, 0.5), xpd = TRUE, cex = 0.7)

seqlegend(ZAU.seq, cpal = colores)
dev.off()

# Cálculo y visualización de estadísticas descriptivas *******Describe******************
dev.new()
par(mfrow = c(2, 2))
seqdplot(ZAU.seq, with.legend = FALSE, border = "grey",cpal = colores); grid(col="black")
seqHtplot(ZAU.seq); grid(col = "red")
seqmsplot(ZAU.seq, with.legend = FALSE, border = "grey",cpal = colores); grid(col="black")
seqmtplot(ZAU.seq, with.legend = FALSE,cpal = colores); grid(col = "red")
dev.off()

# Typology**************************************************************Typology********
dev.new()
dist.om1 <- seqdist(ZAU.seq, method = "OM", indel = 1, sm = "TRATE")
library(cluster)
clusterward1 <- agnes(dist.om1, diss = TRUE, method = "ward")
#library(factoextra)
#fviz_nbclust(dist.om1, FUNcluster = hcut, method = "silhouette")
plot(clusterward1, which.plot = 2)

rect.hclust(clusterward1,k=2,border="red")
cl1.4 <- cutree(clusterward1, k = 2)
cl1.4fac <- factor(cl1.4, labels = paste("Type", 1:2))

seqIplot(ZAU.seq, group = cl1.4fac, sortv = "from.start", border = "black",cpal = colores)

seqdplot(ZAU.seq, group = cl1.4fac, border = "black",cpal = colores)

seqrplot(ZAU.seq, diss = dist.om1, group = cl1.4fac, border = "black",cpal = colores)